<h2> Jonathan Tan's Python YFinance Powered Ticker Parser </h2>

In [1]:
%pip install yfinance

In [2]:
import pandas as pd
import yfinance as yf
import numpy as np

In [3]:
## Existing Capabilities
dir(yf)

['Ticker',
 'Tickers',
 '__all__',
 '__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'base',
 'const',
 'data',
 'download',
 'enable_debug_mode',
 'exceptions',
 'multi',
 'pdr_override',
 'scrapers',
 'set_tz_cache_location',
 'shared',
 'ticker',
 'tickers',
 'utils',
 'version']

<h3> Single Ticker General Information  </h3>

In [4]:
ticker = input("Please enter the ticker that you want to search for")
user_input = input(f"Is {ticker} the stock that you want to search for? Type Yes to continue, No to restart").lower()

if user_input == "yes":
    ticker = ticker  # Use the original ticker
else:
    ticker = input("Please enter the ticker that you want to search for")

cfm_ticker = yf.Ticker(ticker)
# Concatenating financial statements
financials_combined = pd.concat([cfm_ticker.financials, cfm_ticker.balance_sheet, cfm_ticker.cashflow])
financials_combined = financials_combined.T
financials_combined

financials_combined.to_excel(f'Financial_Information_For_{ticker}.xlsx', index= True)

Please enter the ticker that you want to search forTSLA
Is TSLA the stock that you want to search for? Type Yes to continue, No to restartYes


In [5]:
# Getting the ticker from the user
ticker = input("Please enter the ticker that you want to search for: ")
company = yf.Ticker(ticker)

# Concatenating financial statements
financials_combined = pd.concat([company.financials, company.balance_sheet, company.cashflow])

# Asking user for the desired financial metric
metric = input(f"Enter the financial metric you want to compare for {ticker} (e.g., 'Ebitda'): ")

# Checking if the metric exists in the combined DataFrame
if metric in financials_combined.index:
    metric_data = financials_combined.loc[metric]
    print(f"{metric} data for {ticker}:\n", metric_data)
    # Optionally, writing to an Excel file
    metric_data.to_excel(f"{ticker}_{metric}_data.xlsx")
else:
    print(f"Metric '{metric}' not found for {ticker}.")


Please enter the ticker that you want to search for: AAPL
Enter the financial metric you want to compare for AAPL (e.g., 'Ebitda'): EBIT
EBIT data for AAPL:
 2023-09-30    114301000000.0
2022-09-30    119437000000.0
2021-09-30    108949000000.0
2020-09-30     66288000000.0
Name: EBIT, dtype: object


<h2> Getting Multiple Tickers General Information


In [7]:
## Doing the same for multiple tickers
tickers = []

while True:
    ticker = input("Enter a stock ticker, or type 'done' to finish: ")
    if ticker.lower() == 'done':
        break
    tickers.append(ticker)

print("Collected tickers:", tickers)

tickers = [yf.Ticker(ticker) for ticker in tickers]
tickers

dfs = []
for ticker in tickers:
    ## Getting PNL
    pnl = ticker.financials
    bs = ticker.balance_sheet
    cf = ticker.cashflow

    ## Getting financial statement for each ticker
    fs = pd.concat([pnl,bs,cf])

    ## Transpose
    data = fs.T
    data = data.reset_index() ## reset the index data into appropriate col
    data.columns=['Date', *data.columns[1:]] ## Rename the old index to Date
    data['Company Ticker'] = ticker.ticker
    dfs.append(data)


parser = pd.io.parsers.base_parser.ParserBase({'usecols': None})
for df in dfs:
    df.columns = parser._maybe_dedup_names(df.columns)

df = pd.concat(dfs, ignore_index=True)
df = df.set_index(['Company Ticker', 'Date'])
df.to_excel('Multiple_Ticker_Financial_Information.xlsx', index=True)



Enter a stock ticker, or type 'done' to finish: AMZN
Enter a stock ticker, or type 'done' to finish: GOOG
Enter a stock ticker, or type 'done' to finish: TSLA
Enter a stock ticker, or type 'done' to finish: META
Enter a stock ticker, or type 'done' to finish: done
Collected tickers: ['AMZN', 'GOOG', 'TSLA', 'META']


GO<h2> Getting Multiple Ticker Metric Comparison



In [8]:
# Collecting tickers from user
tickers = []
while True:
    ticker = input("Enter a stock ticker, or type 'done' to finish: ")
    if ticker.lower() == 'done':
        break
    tickers.append(ticker)

# Asking user for the desired financial metric
metric = input("Enter the financial metric you want to compare (e.g., 'Ebitda'): ")

# Fetching the specified metric for each ticker
metric_data = {}
for ticker in tickers:
    company = yf.Ticker(ticker)

    # Concatenating financial statements
    financials_combined = pd.concat([company.financials, company.balance_sheet, company.cashflow])

    # Checking if the metric exists in the combined DataFrame
    if metric in financials_combined.index:
        metric_data[ticker] = financials_combined.loc[metric]
    else:
        print(f"Metric '{metric}' not found for {ticker}. Skipping.")

# Creating a DataFrame for the specified metric comparison
metric_df = pd.DataFrame(metric_data)

metric_df.to_excel(f"Multiple_Ticker_{metric}_comparison.xlsx", index=True)


Enter a stock ticker, or type 'done' to finish: AMZN
Enter a stock ticker, or type 'done' to finish: GOOG
Enter a stock ticker, or type 'done' to finish: META
Enter a stock ticker, or type 'done' to finish: TSLA
Enter a stock ticker, or type 'done' to finish: done
Enter the financial metric you want to compare (e.g., 'Ebitda'): EBITDA
